In [7]:
import pyodbc

def check_database_exists(server_name, database_name, username, password, driver="ODBC Driver 17 for SQL Server"):
    """
    Checks if a specific database exists on the SQL Server.

    Args:
        server_name (str): The name of the SQL Server instance.
        database_name (str): The name of the database to check.

    Returns:
        bool: True if the database exists, False otherwise.
    """
    if not all([server_name, database_name, username, password]):
        raise ValueError("All parameters must be provided and non-empty.")

    connection_string = (
        f"DRIVER={{ODBC Driver 17 for SQL Server}};"  # Adjust driver as needed
        f"SERVER={server_name};"
        f"UID={username};"  # Uncomment and provide if not using trusted connection
        f"PWD={password};"  # Uncomment and provide if not using trusted connection
    )

    try:
        cnxn = pyodbc.connect(connection_string)
        cursor = cnxn.cursor()

        # Query to check if the database exists
        query = f"SELECT DB_ID('{database_name}')"
        cursor.execute(query)

        db_id = cursor.fetchone()[0]
        cursor.close()
        cnxn.close()

        return db_id is not None

    except pyodbc.Error as ex:
        sqlstate = ex.args[0]
        print(f"Error connecting to SQL Server: {sqlstate}")
        return False

# Example usage:
server = ''  # Replace with your SQL Server instance name
db_to_check = ''  # Replace with the database name you want to check
username = ''  # Replace with your SQL Server username
password = ''  # Replace with your SQL Server password

if check_database_exists(server, db_to_check, username, password):
    print(f"Database '{db_to_check}' exists on server '{server}'.")
else:
    print(f"Database '{db_to_check}' does NOT exist on server '{server}'.")


ValueError: All parameters must be provided and non-empty.

In [4]:
import dotenv
from openai import AzureOpenAI
import os
def AI_Run(content):
    dotenv.load_dotenv()

    client = AzureOpenAI(
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
        api_version="2025-01-01-preview",
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
    )

    response = client.chat.completions.create(
        model="gpt-5-mini",
        messages=[
            {
                "role": "user",
                "content": content
            }
        ]
    )

    print(response)
    return response  # Return the full response object for AI logging
x = AI_Run("Hello, how are you?")
print(x.choices[0].message.content)
print(x.choices[0].CompletionUsage)  # Full message object

ChatCompletion(id='chatcmpl-CRCLXm0y5IIcyNvwnRniUcah3o7pf', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hi — I’m doing well, thanks! How can I help you today?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1760598351, model='gpt-5-mini-2025-08-07', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=26, prompt_tokens=12, total_tokens=38, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, 

In [12]:
print(x.choices[0].message.content)
print(x.model)  # Full message object
print(x.usage.total_tokens)
print(x.usage.prompt_tokens)
print(x.usage.completion_tokens)

Hi — I’m doing well, thanks! How can I help you today?
gpt-5-mini-2025-08-07
38
12
26


In [20]:
from urllib.parse import quote_plus
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError

# URL encode the connection parameters
encoded_driver = quote_plus("ODBC Driver 17 for SQL Server")
connection_string = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver={encoded_driver}&connection_timeout=30&command_timeout=30"

try:
    print(f"Attempting SQLAlchemy connection to {server}...")
    engine = create_engine(connection_string)
    
    with engine.connect() as connection:
        result = connection.execute(text("SELECT @@VERSION"))
        version = result.fetchone()
        print(f"✓ SQLAlchemy connection successful!")
        print(f"SQL Server version: {version[0]}")
        
        result = connection.execute(text("""
            SELECT TABLE_NAME 
            FROM INFORMATION_SCHEMA.TABLES 
            WHERE TABLE_NAME = 'ConsumptionLogs'
        """))
        table = result.fetchone()
        if table:
            print(f"✓ Table 'ConsumptionLogs' exists")
        else:
            print(f"✗ Table 'ConsumptionLogs' NOT found")
    
except SQLAlchemyError as e:
    print(f"✗ SQLAlchemy connection failed: {e}")
except Exception as e:
    print(f"✗ Unexpected error: {e}")

Attempting SQLAlchemy connection to SPL-VMSQLIT19A...
✗ SQLAlchemy connection failed: (pyodbc.OperationalError) ('08001', '[08001] [Microsoft][ODBC Driver 17 for SQL Server]Named Pipes Provider: Could not open a connection to SQL Server [53].  (53) (SQLDriverConnect); [08001] [Microsoft][ODBC Driver 17 for SQL Server]Login timeout expired (0); [08001] [Microsoft][ODBC Driver 17 for SQL Server]Invalid connection string attribute (0); [08001] [Microsoft][ODBC Driver 17 for SQL Server]A network-related or instance-specific error has occurred while establishing a connection to SQL Server. Server is not found or not accessible. Check if instance name is correct and if SQL Server is configured to allow remote connections. For more information see SQL Server Books Online. (53)')
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [ ]:
import pyodbc

server = "SPL-VMSQLIT19A"
database = "DWHStaging"
username = "Host_ETL_Datamart"
password = "Host_ETL_Datamart@123$%"

# Test 1: Basic connection
connection_string = (
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={username};"
    f"PWD={password};"
    f"Timeout=5;"
)

try:
    print(f"Attempting to connect to {server}...")
    cnxn = pyodbc.connect(connection_string)
    cursor = cnxn.cursor()
    cursor.execute("SELECT @@VERSION")
    version = cursor.fetchone()
    print(f"✓ Connection successful!")
    print(f"SQL Server version: {version[0]}")
    
    # Check if table exists and get schema information
    cursor.execute("""
        SELECT TABLE_SCHEMA, TABLE_NAME 
        FROM INFORMATION_SCHEMA.TABLES 
        WHERE TABLE_NAME = 'ConsumptionLogs'
    """)
    table = cursor.fetchone()
    if table:
        schema_name = table[0]
        table_name = table[1]
        print(f"✓ Table '{schema_name}.{table_name}' exists")
    else:
        print(f"✗ Table 'ConsumptionLogs' NOT found")
        schema_name = "dbo"  # Default schema
    
    # Insert data with schema prefix
    insert_query = f"""
    INSERT INTO {schema_name}.ConsumptionLogs (
    [AIProvider], [Model], [Prompt], [Completion],
    [TokensPrompt], [TokensCompletion], [TokensTotal], [DurationSeconds]
    )
    VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """

    cursor.execute(
        insert_query,
        (
            "hello",
            "hello",
            "hello",
            "hello",
            1,
            1,
            1000,
            0.12,
        )
    )
    cnxn.commit()  # Commit the transaction
    print(f"✓ Data inserted successfully! Rows affected: {cursor.rowcount}")
    
    cursor.close()
    cnxn.close()
except pyodbc.Error as e:
    print(f"✗ Connection failed: {e}")


Attempting to connect to SPL-VMSQLIT19A...
✓ Connection successful!
SQL Server version: Microsoft SQL Server 2019 (RTM-CU25-GDR) (KB5036335) - 15.0.4360.2 (X64) 
	Mar 19 2024 00:23:01 
	Copyright (C) 2019 Microsoft Corporation
	Standard Edition (64-bit) on Windows Server 2019 Datacenter 10.0 <X64> (Build 17763: ) (Hypervisor)

✓ Table 'AI.ConsumptionLogs' exists
✓ Data inserted successfully! Rows affected: 1
